<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/main/20_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install hdwallet py_crypto_hd_wallet cryptos eciespy qrcode

# Our Key Handing Up Until Now

Real drawbacks:

- To send and recieve coins we would have to create a new private key, public key and address. 
- If we wanted to send coins from one address to another we'd have to create another private key, public key and address.
- If we wanted to send to one another we'd have to send each other our addresses.
- If we wanted to create a multisig wallet together we'd have to share our public keys.

Wouldn't it be nice if we could generate a single private key from which we could generate a whole set of private keys in a deterministic fashion?

Wouldn't it be nice if we could share a single public key (associate with the above single private key) from which we could generate the whole set of associated public keys in a deterministic fashion?

**This better world is possible with hierarchical deterministic wallets (HD wallets)**


Hierarchical deterministic wallets, also know as HD-wallets, are the standard in the cryptocurrency world but applicable for any application where we new to generate and share keys in this fashion.  We will learn how to use them appropriately.

https://pypi.org/project/py-crypto-hd-wallet/



# BIP44 Derivation Paths

| Coin | Symbol | Derivation Path |
| :------ | :----: | ----------:|
|**Bitcoin**| BTC |`m/44'/0'/0'/0/0`|
|**Litecoin**| LTC |`m/44'/2'/0'/0/0`|
|**Dogecoin**| DOGE |`m/44'/3'/0'/0/0`|
|**DigiByte**| DGB |`m/44'/20'/0'/0/0`|
|**Zcash**| ZEC |`m/44'/133'/0'/0/0`|
|**BitcoinCash**| BCH |`m/44'/145'/0'/0/0`|
|**BitcoinSV**| BSV | `m/44'/236'/0'/0/0`|

Bitcoin Improvement Proposal (BIP) 44 defines the standard derivation path for wallets which generate Pay-to-Public-Key-Hash (P2PKH) addresses. BIP 44 also defines the prefixes to be used with associated extended keys.

According to BIP 44, wallets which generate P2PKH addresses should use a derivation path beginning with `m/44'/`. This means the first address generated by a mainnet Bitcoin wallet will have a derivation path of `m/44'/0'/0'/0/0`.



`m/44'/0'/0'/0/0` is labeled as `master / purpose / coin_type / account / change / address_index`

- **master** all HD wallets begin with `m` in the first position
- **purpose** all coins adoptin BIP44  use `44`
- **coin_type** is used to differentiate between different coins 
- **account** is used to have a branch point for different subsets of key pairs
- **change** this field is used to differentiate between receipt addresses and change addresses
- **address_index** this final field is used to iterate through different keypairs/addresses

<div>
<img src="https://github.com/ProfDoeg/Colegio_Invisible/raw/main/img/hd_wallet.png"  width="500"/>
</div>

The brilliant feature of this structure is that there is a hierarchy of public and private keys.

With ONE master private key you can control/generate many private keys for many different coins.  In addition you can share ONE public key to share ALL the derived public keys and addresses.

Say you have a business with a point-of-sale cash register that collects DOGE transactions. Each transaction can have a seperate address. You need only share the public key with the `m/44'/3'/0'/0` derivation path. The POS software can then derive the addresses for `m/44'/3'/0'/0/0`, `m/44'/3'/0'/0/1`, `m/44'/0'/0'/0/2`...etc

For our purposes...all the students can generate a single master private key and share one public key with hq. HQ can then access an array of derived public keys and the associated crypto addresses.

# Mnemonic Seed Phrases

BIP39 is the use of a mnemonic phrase -- a group of easy to remember words -- to serve as a back up to recover your wallet and coins in the event your wallet becomes compromised, lost, or destroyed. This is also known as a mnemonic seed (phrase), recovery phrase, wallet back up, master seed, etc. 

<div>
<img src="https://github.com/ProfDoeg/Colegio_Invisible/raw/main/img/mnemonic.png" width="400"/>
</div>

These words aren't just any words. They are pulled from a specific list of 2048 words known as the BIP39 wordlist. Upon start up, wallets that utilize the BIP39 standard will provide you a 12-24 word phrase randomly chosen from the standard BIP39 wordlist.

Mnemonic seeds are a way of storing the root private key in a human readable format.

The `hdwallet.utils.generate_mnemonic()` function uses a two arguments, strength and language. 

- **strength** 128, 160, 192, 224 or 256
- **language** english, french, italian, spanish, chinese_simplified, chinese_traditional, japanese or korean

In [ ]:
from hdwallet.utils import generate_mnemonic
from py_crypto_hd_wallet import HdWalletBipFactory, HdWalletBip44Coins, HdWalletBipChanges
import psutil
import os
import pandas as pd
import ecies
import eth_keys

In [ ]:
secret_phrase=generate_mnemonic(language="english", strength=256)
secret_phrase

In [ ]:
hd_wallet_factory = HdWalletBipFactory(HdWalletBip44Coins.DOGECOIN)

In [ ]:
my_doge_wallet = hd_wallet_factory.CreateFromMnemonic('DD_wallet', secret_phrase)

In [ ]:
my_doge_wallet.Generate(account_idx = 0, change_idx = HdWalletBipChanges.CHAIN_EXT, addr_num = 4)

In [ ]:
my_doge_wallet_dict=my_doge_wallet.ToDict()
my_doge_wallet_dict

In [ ]:
my_doge_wallet_dict.keys()

In [ ]:
my_doge_wallet_dict['address']['address_1']['wif_priv'] #this is the private key to address 1

In [ ]:
my_doge_wallet_dict['address']['address_1']['raw_uncompr_pub'] #this is the public key to address 1

Remember the hierarchy is `master / purpose / coin_type / account / change / address_index`. Each of these nodes has it's own xpub hierarchical public key. Sharing a xpub gives all of the child public keys. In order to share all of the external addresses (not internal change adresses) we share the xpub associated with the `change` node.

Sharing the xpub from a higher node would reveal more public keys. Sharing at the `account` level would reveal the public keys for receive addresses and change addresses (external and internal).

In [ ]:
my_doge_wallet_dict['change_key']['ex_pub'] #this is the xpub key that can be used to generate all the public keys 

In [ ]:
def get_addresses(wallet):
    addys=wallet.ToDict()['address']
    return [ addys[addy]['address'] for addy in addys]

def get_privates(wallet,form='wif'):
    addys=wallet.ToDict()['address']
    if form=='wif':
      return [ addys[addy]['wif_priv'] for addy in addys]
    else:
      return [ addys[addy]['raw_priv'] for addy in addys]
def get_publics(wallet):
    addys=wallet.ToDict()['address']
    return [ addys[addy]['raw_uncompr_pub'] for addy in addys]

In [ ]:
get_addresses(my_doge_wallet)

# File With Some Ex_PubKeys

```text
student,ex_pubkey
00,dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm
01,dgub8tiZR8EbP42Rg4EAn7jYYQRLjCZ2UHg5Bwdb4eASRBvNYwanrTdp8jvN2bHdakaevMj82u9tknJaYvsiaSMpijn7aP2hSsJbLbj8qbuBmZU
02,dgub8tTFHkG1FajRZH2nscUV9166uRMRRAvbeh6zXCctUVgpNwvWpZ16JT1WrSwW46p2hvfPZNCKmb5xQBcUwKBip7iM3tUNMzAkqDDysR1UhbE
03,dgub8tZWJSrqUJ9KDRAQEHU4YH9ToP6f16jLsvi1cP4kA45MRve9PnBM3BPUgPshBoJBS1BebakEw5Qh9W2pAfTWV8YVgSeUfmyjCkScMGabGMK
04,dgub8tP4gsyL1R7U5eZVTDZbA5DF4foatJGq9EikgtaX3rtUp5hZ1QT9CnCxhf1GmceVsH8xjJmzcCMyXdSxekjvQ8M5fZKaNoenMenPDDmPfwM
05,dgub8uqjRZAXa5dvQ2wTaaxTeS45EDd3BbNCo6X1ifCnH2AJvRiH3CsdS3yTqFMvfU3EptHTtDVUb2V7pb5YNaecf5xPkUQ16CWxWpbPLAxo7hL
06,dgub8uTT3NrNwnwMW1pYo4AHR4XSEUf4pZQtHjg1xSuxSBZCvvketHKK3gKcLuiu8BNW9LTgQ5oZs35PT6ToR6xr4Q2PCqfvvoxQ18HSsS2fPcw


09,dgub8seQT7BUFMQfoVNgMG7W9tXhDcLEYxyM6Ntf4k5qN49cMaeRK1iZegYCpQ4RGeEMUxBtFFvhpWqvvk3McuEUojdhPvYy3wWmXPP4u89RcmN
```

The 00 key and 09 key are xpubs to the same wallet. 00 is from change=0 node while 09 is from account=0 node

In [ ]:
ex_00='dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm'

In [ ]:
watch_00=hd_wallet_factory.CreateFromExtendedKey("watcher_00",ex_00)

In [ ]:
watch_00.Generate(account_idx = 0, change_idx = HdWalletBipChanges.CHAIN_EXT, addr_num = 10)

In [ ]:
watch_00.IsWatchOnly()

In [ ]:
watch_00.ToDict()

In [ ]:
get_addresses(watch_00)

This xpub number 9 is from the account 0 node, therefore both internal and external addresses are available

In [ ]:
ex_09='dgub8seQT7BUFMQfoVNgMG7W9tXhDcLEYxyM6Ntf4k5qN49cMaeRK1iZegYCpQ4RGeEMUxBtFFvhpWqvvk3McuEUojdhPvYy3wWmXPP4u89RcmN'
watch_09_ext=hd_wallet_factory.CreateFromExtendedKey("watcher_09",ex_09)
watch_09_int=hd_wallet_factory.CreateFromExtendedKey("watcher_09",ex_09)
watch_09_ext.Generate(account_idx = 0, change_idx = HdWalletBipChanges.CHAIN_EXT, addr_num = 4)
watch_09_int.Generate(account_idx = 0, change_idx = HdWalletBipChanges.CHAIN_INT, addr_num = 4)
print(get_addresses(watch_09_ext))
print('-'*50)
print(get_addresses(watch_09_int))

#Identity


Identity has different compnents and posible constructions. Here we will form identity as a json dictionary. Every key value pair is opt-in. 

```python
identity_dict = { 'name' : 'DrDoeg',
                  'field' : 'Physics', 
                  'twitter' : 't_physics',
                  'hd_pub_0' : {'pub/44/3/0/0':'dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm',
                                 'sig_trans':'HexSignatutureOfTheTransactionThatStoresThis'} }
```


```python
identity_dict = { 'name' : 'DrDoeg',
                  'field' : 'Physics', 
                  'twitter' : 't_physics',
                  'image' : 'aa0c3ea6b38b2238725df2457eb328bbebd7d91ac5241c24de8ea11616c2bb6d',
                  'addr_0' : 'DRdoEgDC7naAn9bsXEHqdCCPaYN7S8i5ro'
                  'hd_pub_0' : {'pub/44/3/0/0':'dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm',
                                 'sig_trans':'d1ec81ffae055329a23787dc0d9e1525af8893ef3f00a7568505f6220a9b82de4147da07d2a00552a12406c708c571869e0bc892266f9a13dc15528b491cc8f101'} }
```

There can be four types of entries:
- declaritive entries: these are names or datapoints which are stated unproven as self identifiers
- social pointers: these are pointers to social media or websites that contain references to the identity transaction. these references serve as proof in the identity handshake.
- public keys: these are references to hd keys or single keys. these entries are dictionaries that contain the a public key reference and a signature from that keypair of the identity transaction
- prev_id: this is a previous identity transaction hash 

There should be different types of trust/proofs implemented. Maybe and account with both keys capable of withdraw. This method of identity construction does allow for redefined personal digital boundaries.

We can also reference other identity transactions. The authenticated dictionary should be constructed publickeys first, then previous identity transactions, then finally declaratives and socials.

# Header

the header is `\xc1\xdd\x00\x01\x1d\x00\x00`

`1d` is the byte for Identity inscriptions

In [ ]:
identity_dict = { 'name' : 'DrDoeg',
                  'field' : 'Physics', 
                  'twitter' : 't_physics',
                  'image' : 'aa0c3ea6b38b2238725df2457eb328bbebd7d91ac5241c24de8ea11616c2bb6d',
                  'addr_0' : {'addr':'DRdoEgDC7naAn9bsXEHqdCCPaYN7S8i5ro',
                              'sig_trans':'d1ec81ffae055329a23787dc0d9e1525af8893ef3f00a7568505f6220a9b82de4147da07d2a00552a12406c708c571869e0bc892266f9a13dc15528b491cc8f101'},
                  'hd_pub_0' : {'pub/44/3/0/0':'dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm',
                                 'sig_trans':'d1ec81ffae055329a23787dc0d9e1525af8893ef3f00a7568505f6220a9b82de4147da07d2a00552a12406c708c571869e0bc892266f9a13dc15528b491cc8f101'} }

identity_dict       

In [ ]:
import json

pack=json.dumps(identity_dict, indent = '\t').encode()
print(len(pack)/80)
print(pack.decode())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp drive/MyDrive/cinv/llaves.zip llaves.zip 
! unzip llaves.zip
! cp drive/MyDrive/cinv/claves.zip claves.zip 
! unzip claves.zip

In [ ]:
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git
! cp Colegio_Invisible/modules/colegio_tools.py colegio_tools.py

In [ ]:
from colegio_tools import *

In [ ]:
import IPython.display
qr=IPython.display.Image('llaves/mi_addr.png')

privKey=import_privKey('llaves/mi_prv.enc','')
pub=import_pubKey('llaves/mi_pub.bin').to_hex()
addr=import_addr('llaves/mi_addr.bin')

display(qr)
display(pub)
display(addr)



In [ ]:
import cryptos
doge=cryptos.Doge()
uts=doge.unspent(addr)
len(uts)

In [ ]:
def only_conf(utxos):
  import cryptos
  doge=cryptos.Doge()
  return [utxo for utxo in utxos if doge.fetchtx(utxo['output'].split(':')[0])['confirmations']>0]


In [ ]:
dd=only_conf( doge.unspent(addr) )

In [ ]:
len(dd)

In [ ]:
doge=cryptos.Doge()

doge_unspent_total=sum([unspent['value'] for unspent in dd])
print(doge_unspent_total)
tip=1_00_000_000
available=doge_unspent_total-tip
n=4
gift=int(available/4)
outputs=[{'value':gift , 'address': addr} for i in range(n)]
setup_tx=doge.mktx(dd,outputs)
gift/100_000_000

In [ ]:
setup_tx

In [ ]:
tip

In [ ]:
doge.signall(setup_tx,privKey.to_hex()[2:])

In [ ]:
tip/(len(cryptos.serialize(setup_tx))/2)/100000

In [ ]:
funding=doge.pushtx(cryptos.serialize(setup_tx))
funding

This is the identity transaction id:

`1566b4b36e938e4dab05718d16d1d3261ad7b40a2ea4ecaa34b52a822bbe4299`

In [ ]:
identity_txn='1566b4b36e938e4dab05718d16d1d3261ad7b40a2ea4ecaa34b52a822bbe4299'

In [ ]:
doge.fetchtx(funding['data']['txid'])['confirmations']

In [ ]:
qr2=IPython.display.Image('claves/hey_addr.png')

privKey2=import_privKey('claves/hey_prv.enc','Beadog666!')
pub2=import_pubKey('claves/hey_pub.bin').to_hex()
addr2=import_addr('claves/hey_addr.bin')

display(qr2)
display(pub2)
display(addr2)

In [ ]:
signa_1=privKey2.sign_msg(b'\x1d\x00\x00'+bytes.fromhex(identity_txn))
signa_1.to_hex()[2:]

In [ ]:
from eth_keys import keys
hd_key=keys.PrivateKey(bytes.fromhex(get_privates(my_doge_wallet,form='hex')[0]))
hd_key

In [ ]:
signa_hd=hd_key.sign_msg(b'\x1d\x00\x00'+bytes.fromhex(identity_txn))
signa_hd.to_hex()[2:]

In [ ]:
identity_dict['addr_0']['sig_trans']=signa_1.to_hex()[2:]
identity_dict['hd_pub_0']['sig_trans']=signa_hd.to_hex()[2:]
identity_dict

In [ ]:
import json

pack=json.dumps(identity_dict, indent = '\t').encode()
print(len(pack)/80)
print(pack.decode())

In [ ]:
header = b'\xc1\xdd\x00\x01\x1d\x00\x00'
text = '|Declaration of Identity|'.encode() 

In [ ]:
n=3*80
chunks = [pack[i:i+n] for i in range(0, len(pack), n)]

In [ ]:
len(chunks[-1])

In [ ]:
package=[header+text]+chunks
package

In [ ]:
class Cadena():

  def __init__(self,prvkey,data,utxo_dct,tip):
    self.data=data
    self.doge=cryptos.Doge()
    self.clip=[self.data[i:i+80] for i in range(0,len(self.data),80) ]
    self.og_len=len(self.clip)
    self.state='CONF'
    self.utxo=utxo_dct
    self.head_utxo=self.utxo
    self.txn_ids=[self.utxo['output'].split(':')[0]]
    self.prv=prvkey
    self.addr=self.doge.privtoaddr(self.prv)
    self.tip=tip
    self.index=0

  def make_tx(self):
    tx = self.doge.mktx([self.head_utxo],[ {'value':self.head_utxo['value']-self.tip , 'address': self.addr}])
    doge_inscribed_serial_tx=mk_opreturn( self.clip[self.index] , cryptos.serialize(tx))
    doge_inscribed_tx=cryptos.deserialize(doge_inscribed_serial_tx)
    self.signed_inscribed_tx=self.doge.signall(doge_inscribed_tx,self.prv)
    self.state='READY'
  
  def broadcast(self):
    self.cast=self.doge.pushtx(self.signed_inscribed_tx)
    cast_txid=self.cast['data']['txid']
    self.txn_ids.append(cast_txid)
    self.head_utxo={'output':cast_txid+':0' ,'value':self.head_utxo['value']-self.tip }
    self.index=self.index+1
    self.state='SENT'

  def update(self):
    if self.doge.fetchtx(self.head_utxo['output'].split(':')[0])['confirmations']:
      self.state='CONF'
      if self.index==self.og_len:
        self.state='DONE'
    

In [ ]:
sparkle_unspents#=doge.unspent(addr) 

In [ ]:
cadenas=[ Cadena(privKey.to_hex()[2:],*datum_utxo,100_000_000)for datum_utxo in zip(package,sparkle_unspents) ]

In [ ]:
len(cadenas)

In [ ]:
import time

height=doge.current_block_height()-1
print(height)
while True:
  check_height=doge.current_block_height()
  if check_height>height:
    height=check_height
    print(height)
    [cad.make_tx() for cad in cadenas if cad.state=='CONF'];
    [cad.broadcast() for cad in cadenas if cad.state=='READY'];
    [cad.update() for cad in cadenas if cad.state=='SENT'];
    [cad.make_tx() for cad in cadenas if cad.state=='CONF'];
    [cad.broadcast() for cad in cadenas if cad.state=='READY'];
    print([cad.state for cad in cadenas])
    if all([cad.state=='DONE' for cad in cadenas]):
      break
  time.sleep(10)

In [ ]:
def get_output_spend_txns(txn_ident):
  import requests
  import json
  r = requests.get(f'https://sochain.com/api/v2/tx/DOGE/{txn_ident}')
  return [out['spent']['txid'] for out in json.loads(r.text)['data']['outputs']]

def get_op_return(txn_ident):
  import requests
  import json
  r = requests.get(f'https://sochain.com/api/v2/tx/DOGE/{txn_ident}')
  outs=json.loads(r.text)['data']['outputs']
  asm=outs[-1]['script_asm']
  return (asm[10:] if 'OP_RETURN' in asm else None),outs[0]['spent']['txid'] if outs[0]['spent'] else None

def get_op_returns(tx_head,prefix=''):
  import time
  time.sleep(0.25)
  op_ret,next_txn=get_op_return(tx_head)
  if (op_ret and next_txn):
    return get_op_returns(next_txn,prefix+op_ret)
  if op_ret==None:
    return prefix
  else:
    return prefix+op_ret

def read_cadenas(txn_ident):
  datalist=[get_op_returns(txn) for txn in get_output_spend_txns(txn_ident)]
  return datalist[0],b''.join([bytes.fromhex(x) for x in datalist[1:] ])



In [ ]:
(hdr,bdy)=read_cadenas('1566b4b36e938e4dab05718d16d1d3261ad7b40a2ea4ecaa34b52a822bbe4299')

In [ ]:
id_dct=json.loads(bdy.decode())
id_dct

In [ ]:
get_privates(my_doge_wallet,form='hex')[0]

In [ ]:
sig=ecies.hex2prv('a243bc0fb06da7f402adaa38ed726e5f08f78465e192065c1efaf3d3a8f21b56').sign(b'testbytesthatwillbe32txnhashbytes')
len(sig.hex())/2

In [ ]:
type(eth_keys.keys.PrivateKey(bytes.fromhex('a243bc0fb06da7f402adaa38ed726e5f08f78465e192065c1efaf3d3a8f21b56')))

In [ ]:
eth_keys.keys.PrivateKey(bytes.fromhex('a243bc0fb06da7f402adaa38ed726e5f08f78465e192065c1efaf3d3a8f21b56')).sign_msg(b'hello').to_bytes()

In [ ]:
ecies.hex2prv('a243bc0fb06da7f402adaa38ed726e5f08f78465e192065c1efaf3d3a8f21b56').sign(b'heyllo').hex()

In [ ]:
ecies.PrivateKey(bytes.fromhex('a243bc0fb06da7f402adaa38ed726e5f08f78465e192065c1efaf3d3a8f21b56')).sign(b'heyllo').hex()

In [ ]:
import cryptos
doge=cryptos.Doge()

In [ ]:
my_doge_wallet.ToDict()

 ```
 def sign_msg(self, message: bytes) -> 'Signature':
        message_hash = keccak(message)
        return self.sign_msg_hash(message_hash)

```


In [ ]:
M=b'new day hey '

In [ ]:
from eth_keys import keys
pk = keys.PrivateKey(b'\x01' * 32)
signature = pk.sign_msg(M)
signature,pk, type(pk)

In [ ]:
pk2=ecies.PrivateKey(b'\x01' * 32)
signature2=pk2.sign(M)
signature2.hex()

In [ ]:
import hashlib
signature2k=pk2.sign(M,hasher=lambda x: hashlib.sha256(x).digest())
signature2k

In [ ]:
signature2k1=pk2.sign(M)
signature2k1

In [ ]:
from eth_utils import keccak
import hashlib
signature2k2=pk2.sign(M,hasher=keccak)
signature2k2.hex()[10:], type(pk2)

In [ ]:
signature2.hex(), signature2k.hex() , signature2k1.hex(), signature2k2.hex() , signature.to_hex()

In [ ]:
signature2k2.hex()[10:][64:68], signature2k2.hex()[10:][:64] + signature2k2.hex()[10:][68:] , signature.to_hex()[2:]

In [ ]:
type(signature2k2)

In [ ]:
type(pk2)

In [ ]:
pk2.public_key.verify(signature2k,M)

In [ ]:
pk2.public_key.verify(signature2k2,M,keccak)

In [ ]:
signature.verify_msg(M,pk.public_key)

In [ ]:
type(signature)

In [ ]:
signature2k2[:5].hex()

In [ ]:
identity_dict = { 'name' : 'DrDoeg',
                  'field' : 'Physics', 
                  'twitter' : 't_physics',
                  'image' : 'aa0c3ea6b38b2238725df2457eb328bbebd7d91ac5241c24de8ea11616c2bb6d'
                  'hd_pub_0' : {'pub/44/3/0/0':'dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm',
                                 'sig_trans':'d1ec81ffae055329a23787dc0d9e1525af8893ef3f00a7568505f6220a9b82de4147da07d2a00552a12406c708c571869e0bc892266f9a13dc15528b491cc8f101'} }

In [ ]:
import json

pack=json.dumps(identity_dict, indent = '\t').encode()
print(len(pack)/80)
print(pack.decode())

In [ ]:
json.loads()